In [31]:
import ast
from pprint import pprint


def parse_dag_file(dag_path):
    """Parse a DAG file and return a dictionary of task definitions."""
    with open(dag_path, "r") as f:
        tree = ast.parse(f.read())

        return tree


tree = parse_dag_file(dag_path="dag_tester.py")

In [32]:
# print(ast.dump(tree))

for node in ast.walk(tree):
    if isinstance(node, ast.Assign):
        # print(node, node.targets)
        for target in node.targets:
            print(target.id)
            print(node.value)

default_args
END_MESSAGE
FILE_PATH
task_start
task_sensor
message
task_cmd
task_end


In [33]:
tasks = {}
candidate_tasks = ["task_cmd", "task_sensor", "end"]

In [34]:
# Find taskflow operators
# A operator is a taskflow operator if it is calling a decorator function
# @task.bash
# def cmd_task(message):
#     return f'echo {message}'
# task_cmd = cmd_task("Hello World")
#
# The follwoing is not a taskflow operator aka traditional operator
# task_sensor = FileSensor(
#     task_id='file_sensor',
#     filepath='test.txt',
#     poke_interval=60,
#     timeout=600,
# )
def find_taskflow_operator(search_id):
    # print(search_id)
    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef) and node.decorator_list is not None:
            # print(ast.dump(node))
            if node.name == search_id:
                # print(ast.dump(node))
                return True, node
            # else:
            #     print(ast.dump(node))

    return False, None


find_taskflow_operator("end")
find_taskflow_operator("cmd_task")

(True, <ast.FunctionDef at 0x7facd3708510>)

In [35]:
def find_traditional_operator(serach_id, node):
    print(serach_id)
    print(ast.dump(node))

In [36]:
# Intention is to replace any global/local references out of the function scope
# so that we can track the actual content of the task
def generate_full_content(operator):
    print(ast.dump(operator))

In [ ]:
for node in ast.walk(tree):
    # Find non-taskflow operators
    if isinstance(node, ast.Assign):
        for target in node.targets:
            if isinstance(target, ast.Name) and target.id in candidate_tasks:
                task_name = target.id
                # tasks[task_name] = node.value
                print(f"Task: {task_name}")
                print(f"Value: {node.value}")
                # print(find_taskflow_operator(task_name))

                # print node value deserialized
                # print(ast.dump(node.value))
                if isinstance(node.value, ast.Call):
                    # print("Function Id:", node.value.func.id)
                    function_name = node.value.func.id
                    # print("Keywords:", node.value.keywords)
                    # print("Args:", node.value.args)

                    (is_taskflow_operator, operator) = find_taskflow_operator(
                        function_name
                    )
                    if is_taskflow_operator:
                        pprint(
                            " ----------------- Taskflow Operator Start -------------------"
                        )
                        pprint(ast.dump(operator))
                        pprint(
                            " ################# Taskflow Operator End ######################"
                        )
                    else:

                        def generate_full_content(operator: ast.Call):
                            content = ""
                            pprint(ast.dump(operator))
                            pprint(operator._fields)
                            pprint(ast.dump(operator.func))
                            for arg in operator.args:
                                print(ast.dump(arg))
                            for keyword in operator.keywords:
                                pprint(ast.dump(keyword))
                                if isinstance(keyword.value, ast.Name):

                                    def find_variable_leaf_node(id: ast.Name):
                                        print("V A L U E", id)
                                        for node in ast.walk(tree):
                                            if isinstance(node, ast.Assign):
                                                for target in node.targets:
                                                    if (
                                                        isinstance(target, ast.Name)
                                                        and target.id == id
                                                    ):
                                                        pprint(ast.dump(node))

                                                        def find_constant_node(
                                                            initial_node: ast.Assign,
                                                        ):
                                                            # recurse until initial_node is not a constant
                                                            # collect all intermediate nodes to hash
                                                            print(
                                                                type(initial_node.value)
                                                            )

                                                        find_constant_node(node)

                                    print(keyword.value)
                                    find_variable_leaf_node(keyword.value.id)
                            # print(ast.dump(operator.keywords))

                        pprint(
                            " ----------------- Traditional Operator Start -------------------"
                        )
                        fn = node.value
                        if isinstance(fn, ast.Call):
                            # if isinstance(fn.func, ast.FunctionDef):
                            # pprint(ast.dump(fn))
                            generate_full_content(fn)
                        pprint(
                            " ################# Traditional Operator End ######################"
                        )
                        # find_traditional_operator(function_name, node.value)

                # Find the content of the task
                # 1. if the task is a traditional function call then print direct
                # 2. if the task is a taskflow operator then find the function call inside the taskflow operator
    elif isinstance(node, ast.FunctionDef) and node.name in candidate_tasks:
        """
        Case: Taskflow Operator
        Scenarion: When didn't assing oprator call to task but direct call of the taskflow operator
        """
        pprint(" ----------------- Taskflow Operator Start -------------------")
        pprint(ast.dump(node))
        pprint(" ################# Taskflow Operator End ######################")

Task: task_sensor
Value: <ast.Call object at 0x7facd3709210>
' ----------------- Traditional Operator Start -------------------'
("Call(func=Name(id='FileSensor', ctx=Load()), args=[], "
 "keywords=[keyword(arg='task_id', value=Constant(value='file_sensor')), "
 "keyword(arg='filepath', value=Name(id='FILE_PATH', ctx=Load())), "
 "keyword(arg='poke_interval', value=Constant(value=60)), "
 "keyword(arg='timeout', value=Constant(value=600)), keyword(arg='sql', "
 "value=Constant(value='SELECT 1;'))])")
('func', 'args', 'keywords')
"Name(id='FileSensor', ctx=Load())"
"keyword(arg='task_id', value=Constant(value='file_sensor'))"
"keyword(arg='filepath', value=Name(id='FILE_PATH', ctx=Load()))"
V A L U E FILE_PATH
("Assign(targets=[Name(id='FILE_PATH', ctx=Store())], "
 "value=Constant(value='/usr/local/airflow/dags/initiator'))")
<class 'ast.Constant'>
"keyword(arg='poke_interval', value=Constant(value=60))"
"keyword(arg='timeout', value=Constant(value=600))"
"keyword(arg='sql', value=Cons

In [38]:
# from graphviz import Digraph

# # Function to add nodes recursively
# def add_node(node, parent=None):
#     node_name = str(id(node)) + str(node.__class__.__name__)
#     dot.node(node_name)
#     if parent:
#         dot.edge(str(id(parent)), node_name)
#     for child in ast.iter_child_nodes(node):
#         add_node(child, node)

# # Create and render Digraph
# dot = Digraph()
# add_node(tree)
# dot.render('my_ast', view=True)